In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

In [14]:
input_file = "output_20200219_2125PM.csv" #complete
#input_file = "output_20191116_1256PM.csv" #partial
df = pd.read_csv(input_file, header = 0)
df['FLOW'] = df['SRCNODE']+'-'+df['DESTNODE']
df['FM']=df['FAILURE']+df['MISSING']
df_failure = df[ (df.FM==1)]

df_dummy = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S'])
y=df_failure['LABEL']
x=df_dummy.drop(['LABEL'],axis=1)

In [15]:
#from sklearn.preprocessing import StandardScaler 
#scaler = StandardScaler()
#scaler.fit(x)
#x = scaler.transform(x)

In [16]:
x.shape[0]

32509

In [39]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-10,hidden_layer_sizes=(10,5), random_state=1)

In [40]:
%%time
clf.fit(x, y)

CPU times: user 45.8 s, sys: 1.66 s, total: 47.5 s
Wall time: 12.1 s


MLPClassifier(activation='relu', alpha=1e-10, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 5), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [32]:
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [41]:
pred=clf.predict(x)
#print(clf.predict_proba(x))

In [42]:
balanced_accuracy_score(y, pred)

0.1908630126696316

In [43]:
f1_score(y, pred,average='weighted')

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.2724583633304553

In [36]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.LABEL==label]
        if test_data_1.shape[0]==0:
            print("No match")
            continue
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        #scaler = StandardScaler()
        #scaler.fit(x_test_1)
        #x_test_1 = scaler.transform(x_test_1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #print(classes[label_index])
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            correct=correct+1
    return correct, correct/num

In [44]:
c,r=accuracy(clf, df_dummy, clf.classes_,3)
c,r

(46, 0.696969696969697)

In [28]:
test_file = "output_20191008_0336AM.csv" #complete
#test_file = "big.csv" #partial
test_df = pd.read_csv(test_file, header = 0)
test_df['FLOW'] = test_df['SRCNODE']+'-'+test_df['DESTNODE']
test_df['FM']=test_df['FAILURE']+test_df['MISSING']
test_df_failure = test_df[ (test_df.FM==1)]

test_df_dummy = pd.get_dummies(test_df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S'])
test_y=test_df_failure['LABEL']
test_x=test_df_dummy.drop(['LABEL'],axis=1)

In [45]:
%%time
c,r=accuracy(clf, test_df_dummy, clf.classes_,3)
c,r

No match
No match
No match
No match
No match
CPU times: user 1.01 s, sys: 10.6 ms, total: 1.02 s
Wall time: 259 ms


(32, 0.48484848484848486)